In [1]:
## SETUP 
imports = ['wrds', 'pandas as pd', 'os', 're', 'pickle', 'numpy as np', 'from name_matching.name_matcher import NameMatcher',
          'from joblib import Parallel, delayed', 'from IPython.display import display, HTML, clear_output',
          'unicodedata','sys', 'matplotlib.pyplot as plt', 'glob', 'shutil','from sklearn.decomposition import PCA']
for command in imports:
    if command.startswith('from'): exec(command)
    else: exec('import ' + command)

if not os.getcwd().endswith('Big Data'):
    os.chdir('../..')
sys.path.append('trade_data_code/2_python')


cluster = 'Google' not in os.getcwd()
if ~cluster:
    raw_admin = '1) data/15_revelio_data/1_inputs/a_raw_data/admin/'
    processed_linkedin = '1) data/15_revelio_data/1_inputs/b_processed_data/linkedin/'
    processed_admin = '1) data/15_revelio_data/1_inputs/b_processed_data/admin/'
if cluster:
    raw_admin = 'data/1_raw_data/admin/'
    processed_linkedin = 'data/2_processed/linkedin/'
    processed_admin = 'data/2_processed/admin/'
import A_helper_functions as hf

In [13]:
## ADD PARENT HQ LOCATION 
subsids =pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet').loc[lambda x: x['is_subsid']]
num_chunks = 50
temp_direct = processed_linkedin + 'temp_5'
os.makedirs(temp_direct, exist_ok=True)
db = wrds.Connection(wrds_username='am0195')
chunks = np.array_split(subsids['ultimate_parent_rcid'].unique(), num_chunks)

# SCRAPE / PROCESS DATA FOR EACH CHUNK 
for index in range(num_chunks):
    file_path = os.path.join(temp_direct, f"temp{index}.parquet")
    if not os.path.exists(file_path):
        clear_output(wait=True)
        print(f"{round(100 * (index + 1) / num_chunks, 2)}%")
        rcid_list = tuple(chunks[index].tolist())
        temp = db.raw_sql(
            """
            SELECT rcid, hq_country
            FROM revelio.company_mapping
            WHERE rcid IN %(rcid_list)s
            """,
            params={"rcid_list": rcid_list}
        )
        temp.to_parquet(file_path)

output = (pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True)
          .drop_duplicates()
          .rename(columns={'rcid': 'ultimate_parent_rcid', 'hq_country' :  'ultimate_parent_hq_country'}))

output = pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet').merge(output, how = 'left')
output.to_parquet('firm_lvl_info_all_matched_firms.parquet')
shutil.rmtree(temp_direct)

In [29]:
## FIND ALL ROLES ASSOCIATED WITH OUR MATCHED COMPANIES
remaining_to_find = (pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet'))

num_chunks = 500
temp_direct = processed_linkedin + 'temp'
os.makedirs(temp_direct, exist_ok=True)
db = wrds.Connection(wrds_username='am0195')
chunks = np.array_split(remaining_to_find['rcid'].unique(), num_chunks)

for index in range(num_chunks):
    file_path = temp_direct + "/temp" + str(index) + ".parquet"
    if not os.path.exists(file_path):    
        clear_output(wait=True)
        print(str(round(100*(index+1)/num_chunks,2))+ '%')
        params = {'rcid_list': tuple(chunks[index].tolist())}
        temp = db.raw_sql(
            """
            SELECT rcid,user_id, weight, total_compensation, startdate, enddate, role_k1500,country, state, metro_area, seniority 
            FROM revelio.individual_positions 
            WHERE rcid IN %(rcid_list)s
            """, 
            params= params)
        for col in ['startdate', 'enddate']:
            temp[col] = pd.to_datetime(temp[col], errors='coerce')
        temp.to_parquet(file_path)

output = pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True)
output.to_parquet(processed_linkedin + 'matched_firm_role_output.parquet')
shutil.rmtree(temp_direct)

In [2]:
## FIND THE PRESTIGE / EDUCATION OF ALL EMPLOYEES ASSOCIATED WITH OUR MATCHED COMPANIES 
num_chunks = 500
temp_direct =  processed_linkedin + 'temp'                              
shutil.rmtree(temp_direct)
os.makedirs(temp_direct, exist_ok=True)
role_output = pd.read_parquet(processed_linkedin + 'matched_firm_role_output.parquet')

existing_prestige = pd.read_parquet(processed_linkedin + 'matched_firm_user_prestige.parquet') ## REMOVE
role_output = role_output.loc[~role_output['user_id'].isin(existing_prestige['user_id'])] ## REMOVE 
                              
chunks = np.array_split(role_output['user_id'].unique(), num_chunks)
db = wrds.Connection(wrds_username='am0195')

for index in range(num_chunks):
    file_path = temp_direct + "/temp" + str(index) + ".parquet"
    if not os.path.exists(file_path): 
        clear_output(wait=True)
        print(str(round(100*(index+1)/num_chunks,2))+ '%')
        temp = db.raw_sql(
            """
            select user_id, prestige, highest_degree 
            from revelio.individual_user 
            where user_id IN %(user_ids)s
            """,
            params= {'user_ids': tuple(chunks[index].tolist())})
        temp.to_parquet(file_path)
    
output = pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True) 
output = pd.concat([output, existing_prestige], ignore_index = True) # REMOVE 
output.to_parquet(processed_linkedin + 'matched_firm_user_prestige.parquet')                              
shutil.rmtree(temp_direct)

55.6%


In [ ]:
## Perform PCA to generate metrics of differentiation 
temp_direct = processed_linkedin + 'temp_role'
os.makedirs(temp_direct, exist_ok=True)

def collapse_year_level(year, making_pca, weight_var, pca_model=None):
    temp = (
        output.assign(
            valid=lambda x: x['startdate'].dt.year.le(year) & 
                            (x['enddate'].isna() | x['enddate'].dt.year.ge(year)),
            wgted_comp=lambda x: x['total_compensation'] * x['weight']
        )
        .loc[lambda x: x['valid']]
        .groupby(['firmid', 'role_k1500'], as_index=False)
        .agg(comp=(weight_var, 'sum'))
        .assign(year=year)
        .pivot_table(index=['firmid', 'year'], columns='role_k1500', values='comp', aggfunc='sum', fill_value=0)
        .pipe(lambda df: df.div(df.sum(axis=1), axis=0))
        .replace([np.inf, -np.inf], np.nan)
        .dropna()
    )
    if making_pca:
        pca_model = PCA(n_components=10)
        pca_model.fit(temp)
        return pca_model
    else:
        file_path = temp_direct + "/temp" + str(year) + ".parquet"
        pd.concat([
            temp.reset_index()[['firmid', 'year']],
            pd.DataFrame(pca_model.transform(temp), columns=[f'{weight_var}_PC{i+1}' for i in range(10)])
        ], axis=1).to_parquet(file_path)
        print(year)
        
#set param values 
years = range(2008, 2024)
sample_year = 2015
   
# Load and merge data
long_data = pd.read_parquet(processed_linkedin + 'matched_firm_role_output.parquet')
matching_output = pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet')[['rcid', 'firmid']]
output = pd.merge(long_data, matching_output)

# run pca analysis
for weight_var in ['wgted_comp','weight']:
    print('starting pca gen')
    pca_model = collapse_year_level(sample_year, True, weight_var)
    print('finished pca gen')
    [collapse_year_level(year,False,weight_var,pca_model) for year in range(2008, 2024)]
    (pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True)
     .to_parquet(processed_linkedin + 'matched_firm_pca_'+ weight_var + '_output.parquet'))
shutil.rmtree(temp_direct)

In [ ]:
########################################################################################
# FIND THE AMOUNT OF WORKERS CURRENTLY WORKING OR WITH EXPERIENCE ABROAD PER COMPANY 
########################################################################################

### SET PARAMETERS
num_chunks = 10000
temp_direct = processed_linkedin + 'temp_4'
os.makedirs(temp_direct, exist_ok=True)
db = wrds.Connection(wrds_username='am0195')
matching_output = pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet')[['rcid', 'firmid']]

role_output = pd.read_parquet(processed_linkedin + 'matched_firm_role_output.parquet').merge(matching_output)
chunks = np.array_split(role_output['firmid'].unique(), num_chunks)
linkedin_to_iso_cross_walk = (pd.read_csv(processed_admin +'linkedin_to_iso_crosswalk.csv')
                              .assign(needs_collapse = lambda df: df.groupby('ctry')['ctry'].transform('count') >1))

### DEFINE FUNCTIONS 
def run_subsection(index):
    clear_output(wait=True)
    print(str(round(100*(index+1)/num_chunks,2))+ '%')
    
    role_subset = role_output.loc[lambda x: x['firmid'].isin(chunks[index])]
    params = {"user_ids":  tuple(role_subset['user_id'].unique())}
    ever_role_subset = db.raw_sql(
        """
        SELECT user_id, country, startdate, enddate
        FROM revelio.individual_positions 
        where user_id IN %(user_ids)s
        """,
        params= params 
    )
    print('done scraping')
    subset_output = pd.concat([collapse_year_level(year, role_subset, ever_role_subset) for year in range(2009, 2021)],
                            ignore_index=True)
    subset_output.to_parquet(temp_direct + "/temp"+str(index)+".parquet")

    
def collapse_year_level(year, role_subset, ever_role_subset):
    print(year)
    cutoff_date = pd.Timestamp(f'{year}-01-01')
    temp = (
        ## determine which users are active in a given year for a given firm
        role_subset
        .rename(columns={'country':'current_ctry'})
        .assign(
                startdate =lambda x: pd.to_datetime(x['startdate'], errors='coerce'),
                enddate =lambda x: pd.to_datetime(x['enddate'], errors='coerce')) 
        .assign( valid = lambda x: (x['startdate'] <= cutoff_date) & ((x['enddate'].isna()) | (x['enddate'] >= cutoff_date)))
        .loc[lambda x: x['valid'], ['firmid','user_id', 'current_ctry', 'weight', 'total_compensation']]
        .assign(priority = lambda x: (x['current_ctry'] == 'France').astype(int))
        .sort_values(by = 'priority', ascending = False)
        .drop_duplicates(subset = ['firmid', 'user_id'], keep = 'first')
    
        ## add all the roles those users have ever held 
         .merge(ever_role_subset).drop_duplicates()
    
        # drop all roles that start after the year of interest
        .assign(startdate =lambda x: pd.to_datetime(x['startdate'], errors='coerce'),
                enddate =lambda x: pd.to_datetime(x['enddate'], errors='coerce'),
               years_since_active = lambda x: (year - x['enddate'].dt.year.fillna(year)).clip(lower=0))
         .loc[lambda x: (x['startdate'] <= cutoff_date) & ~x['country'].eq('France')]
    
        # compute the employee's effective tenure in the foreign market 
        .assign(effective_end_date = lambda x: x['enddate'].where(x['enddate'].notna() 
                                                                  & x['enddate'].lt(cutoff_date), cutoff_date))
        .assign(duration = lambda x: (x['effective_end_date'] - x['startdate']).dt.days/ 365.25)
    
        # collapse down (note this will double count tenure if they held multiple roles 
        .groupby(['firmid', 'user_id', 'current_ctry', 'country'], as_index=False)
        .agg({ 'duration': 'sum','years_since_active': 'min', 'total_compensation' : 'min', 'weight' : 'min'})
        .rename(columns={'country':'ctry'})
        .assign(year = year) 
    )
    return(temp)

### EXECUTE SCRAPING AND INITIAL COLLAPSE TO YEAR-firmid-ctry LEVEL 
[run_subsection(index) for index in range(num_chunks)]

In [17]:
### REMOVE 
linkedin_to_iso_cross_walk = (pd.read_csv(processed_admin +'linkedin_to_iso_crosswalk.csv')
                              .assign(needs_collapse = lambda df: df.groupby('ctry')['ctry'].transform('count') >1))
temp_direct = processed_linkedin + 'temp_4'

output = pd.concat([pd.read_parquet(file) for file in glob.glob(temp_direct + "/*.parquet")],ignore_index = True)

## REMOVE 
output = pd.concat([output, pd.read_parquet(processed_linkedin + 'matched_firm_foreign_emp_history_pt1.parquet')], ignore_index = True)
### Match to ISO-2 CODES AND EXPORT 
output.merge(linkedin_to_iso_cross_walk, how = 'left').to_parquet(processed_linkedin + 'matched_firm_foreign_emp_history.parquet')

shutil.rmtree(temp_direct)

In [18]:
output

,firmid,user_id,current_ctry,ctry,duration,years_since_active,total_compensation,weight,year
0,130002736,24268174.0,France,United Kingdom,2.001369,9.0,69702.68,1.293660,2009
1,130002736,112873685.0,France,Ivory Coast,0.999316,10.0,46917.96,1.371606,2009
2,130002736,112873685.0,France,New Caledonia,3.000684,6.0,46917.96,1.371606,2009
3,130002736,247833618.0,Belarus,Belarus,0.503765,0.0,29273.53,1.465224,2009
4,130002736,300268286.0,France,Australia,5.503080,8.0,64046.10,1.325858,2009
...,...,...,...,...,...,...,...,...,...
28960917,403822919,707018941.0,France,Canada,0.169747,1.0,46835.59,1.353552,2019
28960918,403822919,707018941.0,France,Ireland,0.167009,2.0,46835.59,1.353552,2019
28960919,398282939,199282649.0,France,Denmark,0.082136,1.0,38729.63,1.543775,2020
28960920,403822919,707018941.0,France,Canada,0.169747,2.0,46835.59,1.353552,2020


In [ ]:
columns_to_sum = [col for col in combined_output.columns if 'comp' in col or 'empl' in col]
columns_to_keep = ['firmid', 'year','ctry'] + columns_to_sum

combined_output = pd.concat(
    [combined_output.loc[lambda x: ~x['needs_collapse'], columns_to_keep],
    combined_output.loc[lambda x: x['needs_collapse']].groupby(['firmid', 'year', 'ctry'])[columns_to_sum].sum().reset_index()]
    , axis=0, ignore_index=True)

combined_output.to_parquet(processed_linkedin + 'matched_firm_foreign_employment.parquet')

In [ ]:
########################################################################################
# Generate our compensation / employment datasets at the year level 
########################################################################################

role_dict = pd.read_csv(processed_linkedin + 'revelio_role_dict.csv')
prestige = (pd.read_parquet(processed_linkedin + 'matched_firm_user_prestige.parquet').assign(
    advanced_degree=lambda x: x['highest_degree'].isin(['Master', 'MBA', 'Doctor']).astype('boolean'),
    college=lambda x: (x['advanced_degree'] | x['highest_degree'].isin(['Bachelor'])).astype('boolean'),
    ed_data_avail = lambda x: ~x['highest_degree'].isna()))


int_cols = list((set(role_dict.columns) - {'role_k1500', 'Unnamed: 0'}) | {'french_data','french','abroad'})
matching_output = pd.read_parquet(pd.read_parquet(processed_linkedin + 'firm_lvl_info_all_matched_firms.parquet')[['rcid', 'firmid']])

### Get the firm info at the siren level 
base_vars = ['has_subsid', 'is_subsid', 'is_public', 'has_lei', 'french_hq', 'parent_non_french_hq']
matched_firm_base_info = (
    pd.read_parquet(processed_linkedin + 'matched_firm_base_info.parquet').merge(matching_output)
    .assign(needs_collapse = lambda df: df.groupby('firmid')['firmid'].transform('count') > 1)
)
matched_firm_base_info = pd.concat(
    [matched_firm_base_info.loc[lambda x: ~x['needs_collapse'], ['firmid'] + base_vars],
     matched_firm_base_info.loc[lambda x: x['needs_collapse']].groupby('firmid', as_index=False)[base_vars].max()]
    , axis=0, ignore_index=True)

### generate the base for the role data collapse 
role_data = (
    ## merge together all the component datasets 
    pd.read_parquet(processed_linkedin + 'matched_firm_role_output.parquet')
    .merge(prestige, how = 'left')
    .merge(matching_output)
    .merge(role_dict, how = 'left')
    
    ## generate necessary variables  
     .assign(french= lambda x: x['country'].eq("France"),
         comp =  lambda x: x['total_compensation']*x['weight'])
    .assign(french_data = lambda x: x['data'] & x['french'],
            abroad = lambda x: ~x['french'] & ~x['country'].isna(),
            comp_ed_avail = lambda x: x['ed_data_avail']*x['comp'],
            weight_ed_avail = lambda x: x['ed_data_avail']*x['weight'])
)

### Carry out the collapse 
def collapse_wrapper(year):
    clear_output(wait=True)
    print(year)
    temp = (
        role_data
        .assign(valid = lambda x: x['startdate'].dt.year.le(year) & (x['enddate'].isna() | x['enddate'].dt.year.ge(year)))
        .assign(valid = lambda x: x['valid'] & x['comp'].eq(x.groupby(['user_id', 'firmid', 'country','valid'])['comp'].transform('max')))
        .loc[lambda x: x['valid']]
        .assign(ed_comp_denom = lambda x: (x.groupby('firmid')['comp_ed_avail'].transform('sum').where(lambda s: s != 0)),
                ed_weight_denom = lambda x: (x.groupby('firmid')['weight_ed_avail'].transform('sum').where(lambda s: s != 0)))
        .assign(
            comp_weighted_college = lambda x:         x['college']*x['comp']/ x['ed_comp_denom'] ,          
            comp_weighted_advanced_degree = lambda x: x['advanced_degree']*x['comp']/ x['ed_comp_denom'],  
            weighted_college = lambda x:              x['college']*x['weight'] / x['ed_weight_denom'],       
            weighted_advanced_degree = lambda x:      x['advanced_degree']*x['weight'] / x['ed_weight_denom'],
            comp_weighted_prestige = lambda x:        x['prestige'] * x['comp'] / x.groupby('firmid')['comp'].transform('sum'),
            weighted_prestige      = lambda x:        x['prestige'] * x['weight'] / x.groupby('firmid')['weight'].transform('sum')     
       ))
    prestige_vars = ['prestige','college', 'advanced_degree']
    prestige_vars = [f'comp_weighted_{var}' for var in prestige_vars] + [f'weighted_{var}' for var in prestige_vars] 
    temp_prestige = temp.groupby('firmid', as_index=False)[prestige_vars].sum()

    temp = (temp.melt(id_vars=['firmid', 'comp', 'weight'], value_vars=int_cols,var_name='type',
            value_name='valid_flag')
     .loc[lambda x: x['valid_flag'].eq(1)]
     .groupby(['firmid', 'type'])
     .agg(empl=('weight', 'sum'), comp=('comp', 'sum'))
     .reset_index()
     .pivot(index=['firmid'], columns='type', values=['empl', 'comp']))

    temp.columns = [f'{stat}_{vtype}' for stat, vtype in temp.columns]
    temp = temp.reset_index()
    temp = pd.merge(temp, temp_prestige,  how='outer')
    temp = temp.assign(year = year)
    return(temp)
role_annual_collapsed = pd.concat([collapse_wrapper(year) for year in range(2008, 2024)], ignore_index=True)

cols_to_fill = [col for col in role_annual_collapsed.columns
                if (col.startswith('empl') or col.startswith('comp'))
                and not any(exclude in col for exclude in ['prestige', 'college', 'advanced_degree'])
               ]
role_annual_collapsed[cols_to_fill] = role_annual_collapsed[cols_to_fill].fillna(0)

final_output = (pd.merge(role_annual_collapsed, matched_firm_base_info, how = 'left')
                .merge(pd.read_parquet(processed_linkedin + 'matched_firm_pca_wgted_comp_output.parquet'),how = 'left')
                .merge(pd.read_parquet(processed_linkedin + 'matched_firm_pca_weight_output.parquet'),how = 'left'))
for var in int_cols[1:]: final_output[f'share_comp_{var}'] = final_output[f'comp_{var}'] / final_output['comp_total']
final_output.to_parquet(processed_linkedin + 'matched_firm_empl_and_linkedin_characteristics.parquet')

In [ ]:
########################################################################################
# Move all files to their final location
########################################################################################
for_emp = 'matched_firm_foreign_employment.parquet'
all_char = 'matched_firm_empl_and_linkedin_characteristics.parquet'
roles_in_ctry = 'data_roles_in_all_countries.parquet'
all_matched_ids = 'all_linkedin_matched_firmids_final.parquet'
output_dir = '1) data/15_revelio_data/2_outputs/'

pd.read_parquet(processed_linkedin + for_emp).to_parquet(output_dir + '15a_'+ for_emp) 
pd.read_parquet(processed_linkedin + all_char).to_parquet(output_dir + '15b_'+ all_char) 
pd.read_parquet(processed_linkedin + roles_in_ctry).to_parquet(output_dir + '15c_'+ roles_in_ctry) 
pd.read_parquet(processed_admin + all_matched_ids).to_parquet(output_dir + '15d_'+ all_matched_ids) 


